In [1]:
import pandas as pd
import cdsapi
import netCDF4
from netCDF4 import num2date
import numpy as np
import os

In [2]:
def download_climate_data(year, month, day, area, day_night_ind):
    area_str = '_'.join(str(num) for num in area)
    c = cdsapi.Client()
    file_name = './historical_climate_data/weather-'+area_str+'-'+str(year)+'-'+str(month)+'-'+str(day)+'.nc'
    time = '12:00'
    if day_night_ind == "N":
        time = '22:00'
    c.retrieve('reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'format': 'netcdf',
        'variable': [
            '10m_u_component_of_wind', '10m_v_component_of_wind', '2m_temperature',
            'soil_temperature_level_1', 'soil_temperature_level_2', 'soil_temperature_level_3',
            'soil_temperature_level_4', 'soil_type', 'total_precipitation',
        ],
        'year': year,
        'month': month,
        'day': [str(day)],
        'time': [time],
        'area': area,
    },
    file_name)

    nc = netCDF4.Dataset(file_name, mode='r')
    print(nc.variables['t2m'][:])
    cols = list(nc.variables.keys())
    list_nc = []
    for c in cols:
        list_nc.append(list(nc.variables[c][:]))
    df_nc = pd.DataFrame(list_nc)
    df_nc = df_nc.T
    df_nc.columns = cols
    csv_file_name = './historical_climate_data/csv/weather-'+area_str+'-'+str(year)+str(month)+'-'+str(day)+'.csv'
    df_nc.to_csv(csv_file_name, index = False)

In [3]:
#download_climate_data(2022, 1, 11, [59.7686, -125.092, 49.7686, -115.092], 'D')

In [4]:
def download_satellite_image(year, month, day, area, day_night_ind):
    print("image")

In [5]:
def get_climate_data_for_fire_data(file_name):
    file_path = "./historical_fire_data/" + file_name
    fire_data = pd.read_csv(file_path)
    for modis_data_index, modis_row in fire_data.iterrows():
        date_value = modis_row['acq_date']
        date_split = date_value.split("-")
        #print(type(weather_row['DATE']))
        year = date_split[0]
        month = date_split[1]
        day = date_split[2]
        #print(year + month + day)
        confidence = modis_row['confidence']
        latitude = modis_row['latitude']
        longitude = modis_row['longitude']
        day_night_ind = modis_row['daynight']
        #print(type(confidence))
        if confidence > 70:
            north_coord = latitude+0.25
            south_coord = latitude-0.25
            west_coord = longitude-0.25
            east_coord = longitude+0.25
            area = [north_coord, west_coord, south_coord, east_coord]
            #print(area)
            download_climate_data(year, month, day, area, day_night_ind)

In [ ]:
get_climate_data_for_fire_data('canada_2023.csv')

2024-01-11 13:53:49,806 INFO Welcome to the CDS
2024-01-11 13:53:49,807 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-01-11 13:53:49,894 INFO Request is queued
